In [2]:
import pandas as pd
import numpy as np
from sklearn import model_selection

In [68]:
d = pd.DataFrame({'A':[5,4,6,5,5,6,4,4,4,6],'B':[32,14,5,5,45,34,56,7,78,90]})
d.index=[45,87,89,65,87,21,54,98,87,98]
d

,A,B
45,5,32
87,4,14
89,6,5
65,5,5
87,5,45
21,6,34
54,4,56
98,4,7
87,4,78
98,6,90


In [59]:
k=5

In [60]:
folds = model_selection.KFold(n_splits=k,shuffle=True,random_state=0)

#useful?
#folds.get_n_splits(d)

In [74]:
def kfold_target_encode(x_train,y_train,x_test,cols,k=5,alpha=0):
    x_train['target'] = y_train
    globmean = x_train.target.mean()
#for test data
    for col in cols:
        agged_sum = x_train.groupby(col).target.sum()
        x_test[col+'_encoded'] = x_test[col].map(agged_sum)
        x_test[col+'_encoded'].fillna(globmean,inplace=True)
            
#for train data, need to regularise
    #ensure index is reset to avoid breaking later
    

    x_train.reset_index(drop=True,inplace=True)
    x_train[col+'_encoded']=np.nan
    for col in cols:
        folds = model_selection.KFold(n_splits=k,shuffle=True,random_state=0)
        for compute_ind , map_ind in folds.split(x_train):
            mean = x_train.iloc[compute_ind].groupby(col).target.mean()
            count = x_train.iloc[compute_ind].groupby(col).target.count()
            encoding = (mean*count + globmean*alpha)/(alpha+count)
            x_train.loc[map_ind,col+'_encoded'] = x_train.loc[map_ind].col.map(encoding) 
            x_train[col+'_encoded'].fillna(globmean,inplace=True)
    return x_train.drop('target',axis=1) , x_test

In [ ]:
kfold_target_encode

In [77]:
alpha=1
globmean=1

d.reset_index(drop=True,inplace=True)
d['newcol']=np.nan
for train_ind , test_ind in folds.split(d):
    mean = d.iloc[train_ind].groupby('A').B.mean()
    count = d.iloc[train_ind].groupby('A').B.count()
    encoding = (mean*count + globmean*alpha)/(alpha+count)
    
    #    d.iloc[test_ind]['newcol'] = d.iloc[test_ind].A.map(encoding)
    d.loc[test_ind,'newcol'] = d.loc[test_ind].A.map(encoding) 
    print('NEWFOLD')
    print(d.loc[train_ind])
    print('')
    print(d.loc[test_ind])
    print(encoding)
    print('\n')


NEWFOLD
   A   B  newcol
0  5  32     NaN
1  4  14     NaN
3  5   5     NaN
4  5  45     NaN
5  6  34     NaN
6  4  56     NaN
7  4   7     NaN
9  6  90     NaN

   A   B     newcol
2  6   5  41.666667
8  4  78  19.500000
A
4    19.500000
5    20.750000
6    41.666667
Name: B, dtype: float64


NEWFOLD
   A   B     newcol
0  5  32        NaN
1  4  14        NaN
2  6   5  41.666667
3  5   5        NaN
5  6  34        NaN
6  4  56        NaN
7  4   7        NaN
8  4  78  19.500000

   A   B     newcol
4  5  45  12.666667
9  6  90  13.333333
A
4    31.200000
5    12.666667
6    13.333333
Name: B, dtype: float64


NEWFOLD
   A   B     newcol
0  5  32        NaN
2  6   5  41.666667
3  5   5        NaN
4  5  45  12.666667
5  6  34        NaN
7  4   7        NaN
8  4  78  19.500000
9  6  90  13.333333

   A   B     newcol
1  4  14  28.666667
6  4  56  28.666667
A
4    28.666667
5    20.750000
6    32.500000
Name: B, dtype: float64


NEWFOLD
   A   B     newcol
0  5  32        NaN
1  4  14  28.

In [72]:
d

,A,B,newcol
0,5,32,25.000000
1,4,14,42.500000
2,6,5,62.000000
3,5,5,38.500000
4,5,45,18.500000
5,6,34,47.500000
6,4,56,42.500000
7,4,7,49.333333
8,4,78,25.666667
9,6,90,19.500000
